## Resolução das equações de Laplace ou Poisson por Métodos iterativos

A equação de Poisson a duas dimensões espaciais toma a forma:
$$\frac{\partial ^2 u}{\partial x^2} + \frac{\partial ^2 u}{\partial y^2} = f,$$

em que $u$ representa o campo cujos valores queremos determinar em todo o domínio, e em que $f$ representa um termo fonte (ausente na equação de Laplace). No caso electrostático, representa cargas eléctricas. No caso gravítico representa massas. Pode ainda ser vista como estado estacionário para que tende a equação de difusão ou de calor, que veremos no próximo capítulo.

Na sua forma discretizada, numa grelha rectangular de lados $\Delta x$ e $\Delta y$, a equação toma a forma:

$$\frac{u_{i+1,j}^{n}-2u_{i,j}^{n}+u_{i-1,j}^{n}}{\Delta x^2}+\frac{u_{i,j+1}^{n}-2 u_{i,j}^{n}+u_{i,j-1}^{n}}{\Delta y^2}=f_{i,j}^{n}\qquad (1)$$

(o índice superior $n$ vai indicar a ordem de iteração quando chegarmos aos métodos iterativos, enquanto $i,j$ rotulam os pontos da grelha na direção $x,y$, respectivamente).  
Resolvendo a equação (1) em ordem a $u_{i,j}^{n}$, temos:

$$u_{i,j}^{n}=\frac{(u_{i+1,j}^{n}+u_{i-1,j}^{n})\Delta y^2+(u_{i,j+1}^{n}+u_{i,j-1}^{n})\Delta x^2-f_{i,j}^{n}\Delta x^2\Delta y^2}{2(\Delta x^2+\Delta y^2)}$$

No caso particular, mas comum, de $\Delta x =\Delta y \equiv h$ temos a expressão mais simples:

$$u_{i,j}^{n}=\frac{u_{i+1,j}^{n}+u_{i-1,j}^{n} + u_{i,j+1}^{n}+u_{i,j-1}^{n}}{4}-f_{i,j}^{n}h^2.$$

Ou seja, o novo valor é a média do valor dos quatro vizinhos (Laplace) mais a contribuição do termo fonte(Poisson).

Temos também que considerar condições fronteira. Estas podem tomar várias formas. Condições de Dirichelet correspondem a fixar o valor de $u$ na fronteira. Condições de Neumann correspondem a fixar o valor da derivada normal para fora na fronteira (fisicamente, o fluxo). E condições mistas (Robin) correspondem a prescrever em todos os pontos da fronteira uma combinação linear (ou não-linear!) da forma:

$$\alpha_l u(x,y) + \beta_l \frac{\partial u(x,y)}{\partial x} = \gamma_l(y)$$

para o caso de uma fronteira com $x=\text{const}$, e temos expressões análogas para as fronteiras com $y=\text{const}$.

O nosso sistema de equações pode ser escrito na forma geral de um produto de uma matriz de coeficientes a multiplicar pelo vector de incógnitas, que deve ser igual ao termo fonte, de acordo com a equação (1). Teremos então:

$$ \begin{bmatrix} a_{11} & \cdots &  a_{1N}\\ \vdots & \ddots & \vdots \\ a_{N1} & \cdots &  a_{NN} \end{bmatrix} \times   \left[ \begin{array}{c} u_1  \\ \vdots \\u_N \end{array} \right] =\left[ \begin{array}{cc} b_1 \\ \vdots \\ b_N \end{array} \right] $$  
onde a matriz $\mathbf{A}$ dos coeficientes tem dimensões $N\times N$, (onde $N=n_x\times n_y$) é pentadiagonal e só tem elementos não nulos na diagonal principal, onde valem $-\frac{2}{\Delta x^2}- \frac{2}{\Delta y^2}$; na primeira sobrediagonal e na primeira subdiagonal, onde valem   $\frac{1}{\Delta x^2}$; e na $n_x$-ésima sobrediagonal e na $n_x$-ésima subdiagonal, onde valem $\frac{1}{\Delta y^2}.$ Por seu lado o vector dos termos independentes, $\mathbf{b}$, tem como componentes $f_m,\ m\in {1,\cdots,N}.$ (Isto é verdade para ordenamento por colunas primeiro; se fôr por linhas serão as  $n_y$-ésima sobre/sub-diagonais a ser não nulas).

### Métodos iterativos

Para problemas com poucos nodos da rede, podemos usar métodos directos para resolver o sistema de equações que nos dá $u_{i,j}^{n}$. Em 2D a matriz dos coeficientes (quando ordenamos os pontos seguindo ao longo de cada coluna em série, ou quando seguimos ao longo de cada linha em série!) é uma matriz por bandas. No caso de 1D seria tridiagonal, para a qual temos o (eficiente) método de Thomas. No caso de 2D também há métodos mais eficientes que o método geral. Mas à medida que a dimensão do problema cresce, i.é, $N=n_x \times n_y$ é grande, temos que nos voltar para os métodos iterativos.  

Vamos considerar os vários métodos, e algumas variações. Temos assim os métodos de Jacobi, Gauss-Seidel e SOR (*successive-over-relaxations* ou sobre-relaxações sucessivas). 

Ao ordenar os nodos para que as variáveis $u_{i,j}^{n}$ sejam coordenadas de um vector, podemos escolher fazê-lo de várias maneiras (na realidade, qualquer permutação dos nodos é correcta, mas uma ordenação que não seja sistemática não é nada inteligente!). As mais usadas são *column-wise* ou *row-wise*, ié, percorrer cada coluna da primeira à última linha ($n_x$-ésima) , começando na primeira coluna e acabando na última ($n_y$-ésima), ou percorrendo primeiro linhas, para todas as linhas, por ordem. A escolha é indiferente do ponto de vista de escrever o algoritmo ou até o código, mas não o é do ponto de vista do desempenho do programa ao correr! Isso tem a ver com o modo como cada linguagem de programação guarda valores de um **array** na memória. Mas não nos vamos preocupar com esse problema neste momento.

#### Método de Jacobi
Comecemos pelo método de Jacobi. Neste caso a actualização de $u_{i,j}^{n}$, ié, $u_{i,j}^{n+1}$, é feita exclusivamente à custa dos $u_{i,j}^{n}$, obtidos na iteração anterior:

$\begin{align}
    u_{k}^{n+1} &= \frac{1}{a_{kk}}\left[ b_k - \sum_{\substack{l=1 \\ j\neq i}}^{N} a_{kl}u_l^{n} \right] \tag{3.1}\\
    u_{k}^{n+1} &= u_{k}^{n} + \delta u_{k}^{n}
\tag{3.2}\\ 
    \delta u_k &=\frac{1}{a_{kk}} \left[b_k-\sum^{N}_{l=1}a_{kl}u^m_l\right],\ k=1,2,\dots,N,\ l=0,1,\dots
\tag{3.3}
\end{align}$

Também podemos exprimir este resultado numa forma matricial.
$$ A =\begin{bmatrix} a_{11} & 0 & \cdots &  0 &0\\ 0 & 0 & a_{22} & \cdots &  0\\ \vdots & \vdots &\ddots & \vdots & \vdots \\ 0 & 0 &\cdots & 0 & a_{NN} \end{bmatrix} +
 \begin{bmatrix} 0 & 0 & \cdots & 0 &  0\\a_{21} & 0 & 0& \cdots & 0 \\ \vdots & \vdots &\ddots & \vdots & \vdots \\ a_{N1} & a_{N2} &\cdots & a_{N,N-1} &  0 \end{bmatrix} +
 \begin{bmatrix} 0 & a_{12} & \cdots & a_{1,N-1} &  a_{1N}\\0 & 0 & \cdots & a_{2,N-1} & a_{2N}\\ \vdots & \vdots &\ddots &  0&  a_{N-1,N} \\ 0 & 0 &\cdots & 0 & 0 \end{bmatrix}  = L + D + U$$
 Então podemos escrever $A\mathbf{u}=\mathbf{b}$ como $(D+L+U)\mathbf{u}=\mathbf{b}$ donde $D\mathbf{u}=-(L+U)\mathbf{u}+\mathbf{b}$. Supondo que $D^{-1}$ existe e é dada por:
 $$ D^{-1} = \begin{bmatrix} \frac{1}{a_{11}} & 0 & \cdots &  0 &0\\ 0 & 0 & \frac{1}{a_{22}} & \cdots &  0\\ \vdots & \vdots &\ddots & \vdots & \vdots \\ 0 & 0 &\cdots & 0 & \frac{1}{a_{NN}} \end{bmatrix}
 $$
 temos
 $$\mathbf{u}=-D^{-1}(L+U)\mathbf{u}+D^{-1}\mathbf{b}.$$
 
 A iteração de Jacobi pode então ser escrita na forma matricial como:
 $$ \mathbf{u}^{m+1}=-D^{-1}(L+U)\mathbf{u}^{m}+D^{-1}\mathbf{b}.
 $$
 Como as matrizes têm coeficientes constantes, podemos definir $T\equiv -D^{-1}(L+U)$ e a iteração é simplesmente: $\mathbf{u}^{m+1}= T\mathbf{u}^{m}+D^{-1}\mathbf{b}$. (Se as condições fronteira forem de Dirichelet, $\mathbf{b}$ é constante e podemos também definir um $\mathbf{c} \equiv D^{-1}\mathbf{b}$, vindo $\mathbf{u}^{m+1}= T\mathbf{u}^{m}+ \mathbf{c}.$)

#### Método de Gauss-Seidel
O método de Gauss-Seidel tira partido de dois factos: i) pode-se provar que para certas matrizes (e a dos coeficientes da equação de Laplace/Poisson está nesse lote!) cada actualização de um nodo leva o sistema globalmente para mais perto da solução; e ii) se seguirmos uma ordem de actualização regular (como a ordenação que resulta de percorrer cada coluna de seguida) os nodos já actualizados na presente iteração têm valores "melhores" que os da iteração anterior. O algoritmo pode então ser partido em duas somas, ficando na forma:

$ \begin{align}
u_{k}^{n+1} =& \frac{1}{a_{kk}}\left[ b_k - \sum_{l=1}^{k-1} a_{kl}u_l^{n+1} - \sum_{l=k+1}^{N} a_{kl}u_l^{n}\right] \tag{3.4}\\
=&u_{k}^{n} + \frac{1}{a_{kk}}\left[ b_k - \sum_{l=1}^{k-1} a_{kl}u_l^{n+1} - \sum_{l=k}^{N} a_{kl}u_l^{n}\right] \tag{3.5}\\
=&u_{k}^{n} + \delta_{k}^{n}.\tag{3.6}
\end{align}
$

Para o método de Gauss-Seidel, tendo em conta que os valores já actualizados correspondem à parte superior do vector $\mathbf{u}$, que é multiplicada pela matriz $L$, a forma matricial vem:
$$(D+L)\mathbf{u}^{m+1}= -U\mathbf{u}^{m}+ \mathbf{b},
$$ ou
$$ \mathbf{u}^{m+1}=-(D+L)^{-1}U\mathbf{u}^{m}+ (D+L)^{-1}\mathbf{b}.
$$

#### Método de SOR
O método SOR procura melhorar a método de Gauss-Seidel, aumentando o tamanho da correção por um factor $\omega$. O racional é que se em cada actualização estamos a aproximar-nos da solução, então "esticar" essa correção deve-nos levar mais depressa para a solução. Em geral, isso resulta. Mas nem sempre. Também podemos usar $\omega<1$ para "abrandar" problemas que tendem a não convergir, e nesse caso o método chama-se de *sub-relaxações successivas*.  
Note-se que se $\omega=1$ recuperamos o método de Gauss-Seidel, e se $\omega=0$ recuperamos o método de Jacobi.    
Cada problema tem um $\omega$ óptimo, mas que depende quer da equação, quer da geometria do domínio. Em geral é mais fácil obtê-lo por tentativas (frequentemente é à volta de $1.6$).
$ \begin{align}
u_{k}^{n+1}=& (1-\omega) u_{k}^{n} + \omega \delta_{k}^{n} \tag{3.7}\\
=& (1-\omega) u_{k}^{n} + \frac{\omega}{a_{kk}}\left[ b_k - \sum_{l=1}^{k-1} a_{kl}u_l^{n+1} - \sum_{l=k}^{N} a_{kl}u_l^{n}\right]   \tag{3.8}\\
u_{k}^{n+1}=&(1-\omega)u_{k}^{n} + \frac{\omega}{a_{kk}}\left[ b_k - \sum_{l=1}^{k-1} a_{kl}u_l^{n+1} - \sum_{l=k+1}^{N} a_{kl}u_l^{n}\right]
\tag{3.9}
\end{align}
$

Note-se que no caso das equações de Poisson/Laplace é mais eficaz usar a equação directamente, já que a grande maioria dos coeficientes é núla. Assim escrevemos algo como:
$\begin{align}
    &u_{k}^{m+1}=u_{k}^m+\delta u_{k}
\tag{3.10}\\ 
    &\delta u_{k}=\frac{\omega}{4}
      \left[u^{m+1}_{k-1}+u^{m+1}_{k+1}+u^{m}_{k-n_x}+u^{m}_{k+n_x}-4u^{m}_{i,j}-h^2\cdot
      f_{i,j}\right]
\end{align}
$

Definindo o resíduo a cada iteração como:

$
R_{k}=\left[u^{m+1}_{k-1}+u^{m+1}_{k+1}+u^{m}_{k-n_x}+u^{m}_{k+n_x}-4u^{m}_{k}-h^2\cdot
f_{k}\right]
$
podemos escrever para a equação de Poisson:

$
u_{k}^{m+1}=u_{k}^m+\frac{\omega}{4}R_{k}
$

Para a forma matricial fazemos:
$$(D+ \omega L)\mathbf{u}= \omega\mathbf{b} - \left[ \omega U + (\omega -1)D \right]\mathbf{u},
$$
pelo que a iteração se pode escrever como:
$$ \mathbf{u}^{m+1}=-(D+ \omega L)^{-1}\Big(\omega\mathbf{b} - \big[ \omega U + (\omega -1)D \big]\mathbf{u}^{m} \Big)
$$

No entanto, tendo em conta que $(D+\omega L)$ é triangular, os elementos de $\mathbf{u}^{m+1}$ podem ser calculados sequencialmente por substituição para a frente (forward substitution).


 $$u_i^{k + 1} = ( 1 − \omega ) u_i^{k} + \frac{\omega}{a_{ii}}\Big(b_i -\sum_{j<i} a_{ij}u_j^{k+1} - \sum_{j>i} a_{ij}u_j^{k} \Big) ,\qquad i = 1,2 \ldots , N $$

#### Parâmetro de relaxação óptimo
Num domínio rectangular com $n_x+1$, $n_y+1$ nodos (e o mesmo espaçamento em cada direcção), o valor óptimo teórico de $\omega$ é dado por:

$\omega = \frac{2}{1+\sqrt{1-\rho^2}},
$

onde:

$
\rho = \frac{1}{2}[\cos(\pi/n_x) + \cos(\pi(n_y)].$

Se o domínio não fôr rectangular, há uma estimativa devida a Garabedian:

$\omega = \frac{2}{1+3.014 h/\sqrt{A}},
$

onde $h$ é o passo em ambas as direcções, e $A$ é a área do domínio.

In [1]:
from IPython.display import Image

### Ordenação "Red-black"
![red-black ordering](red-black.png)  
Pontos abertos representam fronteira, ponto interiores são vermelhos ou pretos. Cada um tem apenas vizinhos da cor oposta! Assim na actualização dos valores em cada iteração, podemos actualizar todos os pontos vermelhos à custa dos pontos pretos da iteração anterior.  De seguida actualizamos todos os pontos pretos à custa apenas dos pontos vermelhos já calculados para essa iteração. E podemos fazer cada os procedimento em paralelo (ié, vectorizar pontos vermelhos e pontos pretos).


In [ ]:
# Pseudo-código para red-black

# solução inicial u
u = np.array[...];
 
for k in range(len(numIterations)):
    # pontos vermelhos
    for j in enumerate(y[1:-1]):
        for i = in enumerate(x[1:-1]):
            if (mod((i + j)% 2) == 0)
                u[i, j] = 1/4 * (u[i - 1, j] + u[i + 1, j] + u[i, j - 1] + u[i, j + 1] + h**2*f[i, j])
 
    # pontos  pretos
    for j in enumerate(y[1:-1]):
        for i = in enumerate(x[1:-1]):
            if (mod((i + j)% 2) == 1)
                u[i, j] = 1/4 * (u[i - 1, j] + u[i + 1, j] + u[i, j - 1] + u[i, j + 1] + h**2*f[i, j])


Outra forma é usar máscaras (masks), o que permite usar sub-arrays (à semelhança de slices). Ver, por exemplo, https://hplgit.github.io/fdm-book/doc/pub/book/pdf/fdm-book-4screen.pdf sec 3.6.14, para um exemplo do uso de slices.

#### Critério de paragem da iteração
O critério de paragem deve ser mais apurado que um número prescrito máximo de iterações. Num problema podem ser de mais, noutro não serem suficientes. Podemos tomar uma medida numérica do erro como critério. 
  - absoluto:  $\text{max}(\delta u_{i,j})< \epsilon_a$
  - relativo:  $\text{max}(\delta u_{i,j}/u_{i,j})< \epsilon_r$
  - resíduo
  - outros, como:
     - $\frac{1}{N}\sum_i\sum_j |\delta u_{i,j}|< \epsilon_a$ (N é o número total de incógnitas)
     - $\frac{1}{N}\sum_i\sum_j |\delta u_{i,j}/u_{i,j}| < \epsilon_r$

In [ ]:
# exemplo de critério de paragem por erro relativo
reltol=1.0e-3

omega = 1.5
iteration = 0
rel_res=1.0

# solução por SOR
tic=time.time()
while (rel_res > reltol):
    dTmax=0.0
    for j in range(1,ny+1):
        for i in range(1, nx + 1):
            R = (T[j,i-1]+T[j-1,i]+T[j,i+1]+T[j+1,i]-4.0*T[j,i])
            dT = 0.25*omega*R
            T[j,i]+=dT
            dTmax=np.max([np.abs(dT),dTmax])
        
    rel_res=dTmax/np.max(np.abs(T))
    iteration+=1
    
toc=time.time()